# C.	Model Architecture Selection

The optimal architecture for CNN and DNN models is determined, including veveral architectures are tested to identify the best-performing structure before full-scale training.


## DNN

In [1]:
import os
import pandas as pd
import tensorflow as tf
from dnn_models import *
from tensorflow.keras.utils import to_categorical
from multiprocessing import Pool, Manager
import joblib

2025-03-06 03:37:13.685449: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-06 03:37:13.939113: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741232234.049049    8921 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741232234.074741    8921 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 03:37:14.351449: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [10]:
def group_features(df):
    feature_groups = {}

    for col in df.columns:
        if col.startswith("DNN_"):  # Filtra apenas colunas que começam com "DNN_"
            parts = col.split("_")
            if len(parts) > 2:  # Garante que há pelo menos um conjunto intermediário
                base_feature = "_".join(parts[1:-1])  # Pega todos os conjuntos entre o primeiro e o último
                
                if base_feature not in feature_groups:
                    feature_groups[base_feature] = []
                
                feature_groups[base_feature].append(col)

    return feature_groups

def generate_filename(model_name, feature_group, batch_size, patience, epochs, learning_rate, extension):
    """
    Generates a unique filename based on the given parameters.
    
    Args:
        model_name (str): Name of the model.
        feature_group (str): Name of the feature group.
        batch_size (int): Batch size used for training.
        patience (int): Patience value for early stopping.
        epochs (int): Number of epochs.
        learning_rate (float): Learning rate used for training.
        extension (str): File extension (e.g., ".keras" or ".csv").
        
    Returns:
        str: Generated filename.
    """
    return f"{model_name}_{feature_group}_bs{batch_size}_pat{patience}_ep{epochs}_lr{learning_rate}{extension}"


def train_and_evaluate(params, output_dir):
    """
    Trains and evaluates a single combination of model, feature group, and hyperparameters.
    Saves the trained model and metrics if they don't already exist.
    
    Args:
        params (dict): A dictionary containing all necessary parameters for training and evaluation.
        output_dir (str): Directory where models and metrics will be saved.
        
    Returns:
        None
    """
    # Import TensorFlow and other dependencies inside the function
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
    from tensorflow.keras.optimizers import Adam, RMSprop, SGD
    from tensorflow.keras.callbacks import EarlyStopping
    
    # Unpack parameters
    model_func = params["model_func"]
    X_train = params["X_train"]
    y_train = params["y_train"]
    X_val = params["X_val"]
    y_val = params["y_val"]
    X_test = params["X_test"]
    y_test = params["y_test"]
    input_dim = params["input_dim"]
    num_classes = params["num_classes"]
    epochs = params["epochs"]
    batch_size = params["batch_size"]
    patience = params["patience"]
    learning_rate = params["learning_rate"]
    monitor_metric = params["monitor_metric"]
    model_name = params["model_name"]
    feature_group = params["feature_group"]
    
    # Generate filenames
    model_filename = generate_filename(
        model_name, feature_group, batch_size, patience, epochs, learning_rate, ".keras"
    )
    metrics_filename = generate_filename(
        model_name, feature_group, batch_size, patience, epochs, learning_rate, ".csv"
    )
    
    # Define paths
    model_path = os.path.join(output_dir, "models", model_filename)
    metrics_path = os.path.join(output_dir, "metrics", metrics_filename)
    
    # Check if model and metrics already exist
    if os.path.exists(model_path) and os.path.exists(metrics_path):
        #print(f"Skipping training for {model_filename} (already exists)")
        return
    
    # Create and compile the model
    model, optimizer = model_func(input_dim=input_dim, num_classes=num_classes, learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Define early stopping callback
    early_stopping = EarlyStopping(
        monitor=monitor_metric, patience=patience, restore_best_weights=True
    )
    
    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping],
        verbose=0
    )
    
    # Evaluate the model on the test set
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    
    # Save the trained model
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    model.save(model_path)
    print(f"Saved model to {model_path}")
    
    # Save the metrics
    metrics = {
        "Model": model_name,
        "Feature Group": feature_group,
        "Batch Size": batch_size,
        "Patience": patience,
        "Epochs": epochs,
        "Learning Rate": learning_rate,
        "Monitor Metric": monitor_metric,
        "Test Accuracy": accuracy
    }
    os.makedirs(os.path.dirname(metrics_path), exist_ok=True)
    metrics_df = pd.DataFrame([metrics])
    metrics_df.to_csv(metrics_path, index=False)
    print(f"Saved metrics to {metrics_path}")


def test_dnn_models(output_dir, train_df, val_df, test_df, num_workers=10):
    """
    Trains and evaluates multiple DNN models with different hyperparameter combinations,
    saving the results to CSV files using parallel processing.
    
    Args:
        output_dir (str): Directory where result CSV files will be saved.
        train_df (pd.DataFrame): Training dataset.
        val_df (pd.DataFrame): Validation dataset.
        test_df (pd.DataFrame): Test dataset.
        num_workers (int): Number of parallel workers.
        
    Returns:
        None
    """
    # Define the hyperparameters to be tested
    epochs_list = [50, 100]
    batch_sizes = [64, 128]
    patience_list = [3, 5, 10]
    learning_rates = [0.001, 0.0005, 0.0001]
    monitor_metric = 'val_accuracy'
    
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Group features based on their base feature names
    feature_groups = group_features(train_df)
    
    # Define the DNN models to be tested
    models = {
        "DNN-0": create_model_0,
        "DNN-1": create_model_1,
        "DNN-2": create_model_2,
        "DNN-3": create_model_3,
        "DNN-4": create_model_4,
        "DNN-5": create_model_5,
        "DNN-6": create_model_3,
        "DNN-7": create_model_4,
        "DNN-8": create_model_5,
        "DNN-9": create_model_5,
    }
    
    # Determine the number of unique classes
    num_classes = len(train_df['emotion'].unique())  # Determine the number of unique classes
    
    # Use raw integer targets instead of one-hot encoding
    y_train = train_df['emotion'].values
    y_val = val_df['emotion'].values
    y_test = test_df['emotion'].values
    
    # Prepare all combinations of experiments
    experiments = []
    for model_name, model_func in models.items():
        for feature_group, columns in feature_groups.items():
            # Extract features for training, validation, and testing
            X_train = train_df[columns].values
            X_val = val_df[columns].values
            X_test = test_df[columns].values
            
            # Get the number of input features dynamically
            input_dim = X_train.shape[1]
            
            # Iterate over all hyperparameter combinations
            for epochs in epochs_list:
                for batch_size in batch_sizes:
                    for patience in patience_list:
                        for learning_rate in learning_rates:
                            experiments.append({
                                "model_func": model_func,
                                "X_train": X_train,
                                "y_train": y_train,
                                "X_val": X_val,
                                "y_val": y_val,
                                "X_test": X_test,
                                "y_test": y_test,
                                "input_dim": input_dim,
                                "num_classes": num_classes,
                                "epochs": epochs,
                                "batch_size": batch_size,
                                "patience": patience,
                                "learning_rate": learning_rate,
                                "monitor_metric": monitor_metric,
                                "model_name": model_name,
                                "feature_group": feature_group
                            })
    
    # Use multiprocessing to run experiments in parallel
    with Pool(processes=num_workers) as pool:
        # Map the experiments to the worker function
        pool.starmap(train_and_evaluate, [(exp, output_dir) for exp in experiments])

    print("All experiments completed. Models and metrics saved individually.")

In [2]:
import os
import pandas as pd

def process_model_metrics(output_dir):
    """
    Processes all CSV files containing model evaluation metrics from <output_dir>/metrics,
    aggregates them, and saves the consolidated results to two CSV files in <output_dir>.
    
    Args:
        output_dir (str): Directory containing the 'metrics' folder with individual metric files.
    
    Returns:
        None: The function saves the processed results to two CSV files in the specified directory.
    """
    # Define paths
    metrics_dir = os.path.join(output_dir, "metrics")
    consolidated_no_features_path = os.path.join(output_dir, "consolidated_no_features.csv")
    consolidated_total_path = os.path.join(output_dir, "consolidated_total.csv")
    
    # Check if the metrics directory exists
    if not os.path.exists(metrics_dir):
        raise FileNotFoundError(f"The directory '{metrics_dir}' does not exist.")
    
    # Get all CSV files in the metrics directory
    file_paths = [os.path.join(metrics_dir, f) for f in os.listdir(metrics_dir) if f.endswith(".csv")]
    if not file_paths:
        raise ValueError(f"No CSV files found in '{metrics_dir}'.")
    
    # Initialize DataFrame for consolidation
    df_total = pd.DataFrame()
    
    # Process each CSV file
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        df_total = pd.concat([df_total, df], ignore_index=True)  # Append all data to a single DataFrame
    
    # Save raw consolidated data
    df_total = df_total.sort_values(by='Test Accuracy', ascending=False)
    df_total.to_csv(consolidated_total_path, index=False)
    print(f"Saved consolidated total results to {consolidated_total_path}")
    
    # Remove 'Feature Group' column if it exists
    df_no_features = df_total.drop(columns=['Feature Group'], errors='ignore')
    
    # Aggregate data by averaging Test Accuracy across all feature groups for each model-hyperparameter combination
    df_grouped_total = df_no_features.groupby([
        'Model', 'Batch Size', 'Patience', 'Epochs', 'Learning Rate', 'Monitor Metric'
    ], as_index=False).mean()

    # Sort the grouped results by Test Accuracy in descending order
    df_grouped_total = df_grouped_total.sort_values(by='Test Accuracy', ascending=False)

    # Save aggregated results
    df_grouped_total.to_csv(consolidated_no_features_path, index=False)
    print(f"Saved consolidated results without Feature Group to {consolidated_no_features_path}")



## Train one model per feature. Collect metrics

In [12]:
DNN_train = joblib.load("DNN_train.joblib")
DNN_val = joblib.load("DNN_val.joblib")
DNN_val2 = joblib.load("DNN_val2.joblib")

In [4]:
DNN_SELECTION_DIR = "DNN_SELECTION"

In [14]:
test_dnn_models(DNN_SELECTION_DIR, DNN_train, DNN_val, DNN_val2, num_workers=60)

All experiments completed. Models and metrics saved individually.


In [5]:
process_model_metrics(DNN_SELECTION_DIR)

Saved consolidated total results to DNN_SELECTION/consolidated_total.csv
Saved consolidated results without Feature Group to DNN_SELECTION/consolidated_no_features.csv


## Select the best model with the best mean accuracy among all features

In [6]:
results_metrics = pd.read_csv(os.path.join(DNN_SELECTION_DIR, "consolidated_no_features.csv"))
results_metrics.head(1) 

,Model,Batch Size,Patience,Epochs,Learning Rate,Monitor Metric,Test Accuracy
0,DNN-6,64,10,100,0.0005,val_accuracy,0.30353
